In [38]:
from models.db_manager import query_db
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Importez la fonction query_db depuis db_manager.py

# Requête SQL pour sélectionner les données de la table économie
query_economie = "SELECT * FROM economie;"
df_economie = query_db(query_economie)

# Requête SQL pour sélectionner les données du premier tour (ou du second tour si nécessaire)
query_election = "SELECT code_postal, \"% Voix/Ins 1\", \"% Voix/Ins 2\" FROM election_2022_t2"
df_election = query_db(query_election)

# Supprimer les colonnes non désirées dans df_economie
df_economie.drop(['id_economie'], axis=1, inplace=True)

# Fusionner les données économiques avec les données de l'élection sur la colonne code_postal
df_merged = pd.merge(df_economie, df_election, on='code_postal')

# Sélectionner uniquement les données pour La République En Marche
df_party = df_merged

# Séparer les features (X) et la target (y)
X = df_party.drop(['code_postal', '% Voix/Ins 1', '% Voix/Ins 2'], axis=1)
y = df_party['% Voix/Ins 1']

# Diviser les données en ensemble d'entraînement et ensemble de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entraîner le modèle de régression linéaire sur l'ensemble d'entraînement
model = LinearRegression()
model.fit(X_train, y_train)

# Prédire les résultats sur l'ensemble de test pour évaluer la performance du modèle
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print("Erreur quadratique moyenne sur l'ensemble de test:", mse)

# Utiliser le modèle entraîné pour prédire le vote de La République En Marche dans le département 89
dept_89_economie = df_economie[df_economie['code_postal'] == 89].drop('code_postal', axis=1)
prediction_dept_89 = model.predict(dept_89_economie)
print("Prédiction du vote de La République En Marche dans le département 89:", prediction_dept_89[0])


/Users/noodles/workspace/data-science/mspr/models/db_manager.py:27: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(query, conn)


Erreur quadratique moyenne sur l'ensemble de test: 64.96468260708079
Prédiction du vote de La République En Marche dans le département 89: 31.58077046079928
